# AIVIA — Sales CRM Demo (SQL/CSV → Graph → NL-style Questions → Traceable Cypher)

This notebook loads the **Sales CRM synthetic CSVs**, builds a **Neo4j graph**, and answers a few **canonical questions** with **clear Cypher**.

**Flow:** CSVs → Neo4j nodes/edges → Cypher queries → results table  
**Data:** `examples/sales_crm_demo/*.csv` (synthetic, safe for OSS)


In [1]:
# Minimal deps for this demo
%pip -q install neo4j pandas python-dateutil

import os
from pathlib import Path
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import date
from neo4j import GraphDatabase

# ── Configure paths & Neo4j connection ──────────────────────────────────────────
ROOT = Path.cwd().resolve()
DATA_DIR = ROOT.parent / "examples" / "sales_crm_demo" if (ROOT.name == "notebooks") else ROOT / "examples" / "sales_crm_demo"

NEO4J_URI  = os.getenv("AIVIA_NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("AIVIA_NEO4J_USER", "neo4j")
NEO4J_PASS = os.getenv("AIVIA_NEO4J_PASS", "password")

print("DATA_DIR:", DATA_DIR)
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS))
with driver.session() as s:
    s.run("RETURN 1")
print("Connected to Neo4j ✅")


Note: you may need to restart the kernel to use updated packages.
DATA_DIR: /Users/sunnyzheng/AIVIA_Open/AIVIA-core/examples/sales_crm_demo
Connected to Neo4j ✅


In [2]:
def read_csv(name):
    path = DATA_DIR / name
    if not path.exists():
        raise FileNotFoundError(f"Missing {name} at {path}")
    return pd.read_csv(path)

accounts   = read_csv("accounts.csv")
contacts   = read_csv("contacts.csv")
deals      = read_csv("deals.csv")
activities = read_csv("activities.csv")
users      = read_csv("users.csv")

# Optional sets
campaigns  = read_csv("campaigns.csv")  if (DATA_DIR / "campaigns.csv").exists() else pd.DataFrame()
touches    = read_csv("touches.csv")    if (DATA_DIR / "touches.csv").exists() else pd.DataFrame()

print("Loaded CSVs ✅")
display(
    accounts.head(3),
    contacts.head(3),
    deals.head(3),
    activities.head(3),
    users.head(3)
)
if not campaigns.empty:
    display(campaigns.head(3))
if not touches.empty:
    display(touches.head(3))


Loaded CSVs ✅


,account_id,name,industry,region
0,AC-1001,BlueOak Analytics,EdTech,NAMER
1,AC-1002,Nimbus Labs,SaaS,APAC
2,AC-1003,Quasar Health,Healthcare,NAMER


,contact_id,account_id,name,title,email,role
0,CT-2001,AC-1003,Indy Morgan,Finance Manager,indy.morgan@example.com,Procurement
1,CT-2002,AC-1003,Devin Rossi,Security Analyst,devin.rossi@example.com,Security
2,CT-2003,AC-1003,Alex Diaz,Head of Procurement,alex.diaz@example.com,Security


,deal_id,account_id,name,amount,stage,created_date,close_date,owner_user_id,source,is_commit
0,DL-3001,AC-1010,Platform Subscription,12000,Evaluate,2025-07-01,NaN,U-01,Referral,False
1,DL-3002,AC-1006,Enterprise Suite,18000,Prospecting,2025-09-05,NaN,U-01,CMP-4004,False
2,DL-3003,AC-1006,Data Integration,7500,Prospecting,2025-10-09,NaN,U-04,CMP-4003,True


,activity_id,deal_id,type,date,next_step_date
0,AT-5001,DL-3001,Meeting,2025-07-07,NaN
1,AT-5002,DL-3001,Email,2025-09-06,2025-10-25
2,AT-5003,DL-3002,Meeting,2025-09-09,NaN


,user_id,name,team,region
0,U-01,Avery Shaw,Enterprise,NAMER
1,U-02,Jordan Lee,Enterprise,EMEA
2,U-03,Taylor Kim,Mid-Market,NAMER


,campaign_id,name,channel
0,CMP-4001,Q3 Webinar: Data ROI,Webinar
1,CMP-4002,Summer Product Tour,Event
2,CMP-4003,Search Ads - Enterprise,Ads


,touch_id,campaign_id,contact_id,date
0,TC-9001,CMP-4001,CT-2007,2025-07-15
1,TC-9002,CMP-4002,CT-2029,2025-07-08
2,TC-9003,CMP-4003,CT-2017,2025-09-06


## Build the Neo4j graph

We MERGE nodes and relationships so the load is **idempotent**:
- (:Account)<-[:BELONGS_TO]-(:Contact)
- (:Account)-[:HAS_DEAL]->(:Deal)
- (:Deal)-[:HAS_ACTIVITY]->(:Activity)
- (:Deal)-[:OWNED_BY]->(:User)
- Optional: (:Deal)-[:SOURCED_BY]->(:Campaign), (:Campaign)-[:TOUCHED]->(:Contact)


In [10]:
import numpy as np
import pandas as pd

def sanitize_df(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty:
        return df
    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    df = df.where(pd.notnull(df), None)  # NaN -> None

    if "amount" in df.columns:
        df["amount"] = pd.to_numeric(df["amount"], errors="coerce").fillna(0).astype(float)

    # normalize booleans
    if "is_commit" in df.columns:
        df["is_commit"] = df["is_commit"].map(
            lambda v: True if str(v).strip().lower() in {"true","1","yes"} else False
        )

    # keep date-like fields as strings or None
    for c in {"created_date","close_date","date","next_step_date"} & set(df.columns):
        df[c] = df[c].map(lambda v: None if v in (None, "", "nan", np.nan) else str(v))

    return df

accounts   = sanitize_df(accounts)
contacts   = sanitize_df(contacts)
deals      = sanitize_df(deals)
activities = sanitize_df(activities)
users      = sanitize_df(users)
campaigns  = sanitize_df(campaigns) if not campaigns.empty else campaigns
touches    = sanitize_df(touches)   if not touches.empty   else touches

for name, df in [("accounts",accounts),("contacts",contacts),("deals",deals),
                 ("activities",activities),("users",users)]:
    print(name, "rows:", len(df))


accounts rows: 12
contacts rows: 36
deals rows: 17
activities rows: 45
users rows: 6


In [11]:
def load_sales_crm_graph(
    driver,
    accounts, contacts, deals, activities, users,
    campaigns=pd.DataFrame(), touches=pd.DataFrame(),
    clear_first=True
):
    with driver.session() as s:
        if clear_first:
            s.run("MATCH (n) DETACH DELETE n")

        # Accounts
        for r in accounts.to_dict(orient="records"):
            s.run("""
                MERGE (a:Account {id:$id})
                SET a.name=$name, a.industry=$industry, a.region=$region
            """, id=r["account_id"], name=r["name"], industry=r.get("industry"), region=r.get("region"))

        # Users
        for r in users.to_dict(orient="records"):
            s.run("""
                MERGE (u:User {id:$id})
                SET u.name=$name, u.team=$team, u.region=$region
            """, id=r["user_id"], name=r["name"], team=r.get("team"), region=r.get("region"))

        # Contacts + BELONGS_TO
        for r in contacts.to_dict(orient="records"):
            s.run("""
                MERGE (c:Contact {id:$id})
                SET c.name=$name, c.title=$title, c.email=$email, c.role=$role
            """, id=r["contact_id"], name=r.get("name"), title=r.get("title"),
                 email=r.get("email"), role=r.get("role"))
            s.run("""
                MATCH (c:Contact {id:$cid}), (a:Account {id:$aid})
                MERGE (c)-[:BELONGS_TO]->(a)
            """, cid=r["contact_id"], aid=r["account_id"])

        # Deals + edges (store dates as strings)
        for r in deals.to_dict(orient="records"):
            s.run("""
                MERGE (d:Deal {id:$id})
                SET d.name=$name,
                    d.amount=toFloat($amount),
                    d.stage=$stage,
                    d.created_date=$created,    // string
                    d.close_date=$closed,       // string or null
                    d.is_commit=$is_commit,
                    d.source=$source
            """, id=r["deal_id"], name=r.get("name"),
                 amount=r.get("amount", 0.0), stage=r.get("stage"),
                 created=r.get("created_date"), closed=r.get("close_date"),
                 is_commit=bool(r.get("is_commit", False)), source=r.get("source"))
            s.run("""
                MATCH (a:Account {id:$aid}), (d:Deal {id:$did})
                MERGE (a)-[:HAS_DEAL]->(d)
            """, aid=r["account_id"], did=r["deal_id"])
            s.run("""
                MATCH (u:User {id:$uid}), (d:Deal {id:$did})
                MERGE (d)-[:OWNED_BY]->(u)
            """, uid=r["owner_user_id"], did=r["deal_id"])

        # Activities (store dates as strings)
        for r in activities.to_dict(orient="records"):
            s.run("""
                MERGE (ac:Activity {id:$id})
                SET ac.type=$type,
                    ac.date=$when,              // string
                    ac.next_step_date=$next     // string or null
            """, id=r["activity_id"], type=r.get("type"),
                 when=r.get("date"), next=r.get("next_step_date"))
            s.run("""
                MATCH (d:Deal {id:$did}), (ac:Activity {id:$aid})
                MERGE (d)-[:HAS_ACTIVITY]->(ac)
            """, did=r["deal_id"], aid=r["activity_id"])

        # Campaigns / Touches (optional)
        if campaigns is not None and not campaigns.empty:
            for r in campaigns.to_dict(orient="records"):
                s.run("""
                    MERGE (c:Campaign {id:$id})
                    SET c.name=$name, c.channel=$channel
                """, id=r["campaign_id"], name=r.get("name"), channel=r.get("channel"))

        if touches is not None and not touches.empty:
            for r in touches.to_dict(orient="records"):
                s.run("""
                    MATCH (c:Campaign {id:$cid}), (ct:Contact {id:$contact})
                    MERGE (c)-[:TOUCHED {date:$date}]->(ct)
                """, cid=r["campaign_id"], contact=r["contact_id"], date=r.get("date"))

load_sales_crm_graph(driver, accounts, contacts, deals, activities, users, campaigns, touches, clear_first=True)
print("Graph loaded ✅ (string-date loader)")


Graph loaded ✅ (string-date loader)


## Question 1
**“Which open deals > $10k created in the last 60 days have no upcoming meeting in the next 14 days?”**

This is a **missing-edge** / **absent-future-activity** pattern.


In [12]:
q1 = """
MATCH (a:Account)-[:HAS_DEAL]->(d:Deal)
WHERE d.stage <> "Closed Won" AND d.stage <> "Closed Lost"
  AND d.amount > 10000
  AND date(d.created_date) >= date() - duration('P60D')
  AND NOT EXISTS {
    MATCH (d)-[:HAS_ACTIVITY]->(act2:Activity)
    WHERE act2.next_step_date IS NOT NULL
      AND date(act2.next_step_date) <= date() + duration('P14D')
  }
OPTIONAL MATCH (d)-[:OWNED_BY]->(u:User)
RETURN a.name AS account, d.id AS deal_id, d.name AS deal, d.amount AS amount,
       d.stage AS stage, d.created_date AS created, u.name AS owner
ORDER BY amount DESC
"""

with driver.session() as s:
    rows = s.run(q1).data()
pd.DataFrame(rows)


,account,deal_id,deal,amount,stage,created,owner
0,Aurora Retail,DL-3002,Enterprise Suite,18000.0,Prospecting,2025-09-05,Avery Shaw
1,NovaSecure,DL-3010,Platform Subscription,18000.0,Legal,2025-09-22,Avery Shaw
2,BlueOak Analytics,DL-3012,Compliance Module,12000.0,Legal,2025-09-15,Jordan Lee


## Question 2
**“Which commit deals this quarter are missing Finance OR Security in the buying committee?”**

We’ll approximate committee membership as **contacts on the account** (simple demo).  
(If you later model `(:Contact)-[:PLAYS_ROLE]->(:Deal)`, swap the match accordingly.)


In [13]:
q2 = """
// Commit deals in the current quarter that are missing Finance OR Security contacts
WITH date() AS today
WITH today,
     date({year: today.year, month: ((toInteger((today.month-1)/3)*3)+1), day:1}) AS q_start
MATCH (a:Account)-[:HAS_DEAL]->(d:Deal)
WHERE d.is_commit = true
  AND date(d.created_date) >= q_start
  AND d.stage <> "Closed Won" AND d.stage <> "Closed Lost"
OPTIONAL MATCH (a)<-[:BELONGS_TO]-(c_fin:Contact {role:"Finance"})
OPTIONAL MATCH (a)<-[:BELONGS_TO]-(c_sec:Contact  {role:"Security"})
WITH a, d, c_fin, c_sec
WHERE c_fin IS NULL OR c_sec IS NULL
RETURN a.name AS account, d.id AS deal_id, d.name AS deal,
       CASE WHEN c_fin IS NULL THEN "Missing Finance" ELSE "" END +
       CASE WHEN c_fin IS NULL AND c_sec IS NULL THEN " & " ELSE "" END +
       CASE WHEN c_sec IS NULL THEN "Missing Security" ELSE "" END AS gap
ORDER BY account
"""

with driver.session() as s:
    rows = s.run(q2).data()
pd.DataFrame(rows)


,account,deal_id,deal,gap
0,Aurora Retail,DL-3003,Data Integration,Missing Finance & Missing Security


## Question 3
**“Which deals have been in Evaluate for > 21 days and have no activity in 14 days?”**

(If you later add a `StageHistory` node, stage-age becomes even cleaner. For now we use created date as a proxy.)


In [14]:
q3 = """
MATCH (a:Account)-[:HAS_DEAL]->(d:Deal)
WHERE d.stage = "Evaluate"
  AND date(d.created_date) <= date() - duration('P21D')
  AND NOT EXISTS {
    MATCH (d)-[:HAS_ACTIVITY]->(act:Activity)
    WHERE date(act.date) >= date() - duration('P14D')
  }
RETURN a.name AS account, d.id AS deal_id, d.name AS deal, d.created_date AS created
ORDER BY created ASC
"""

with driver.session() as s:
    rows = s.run(q3).data()
pd.DataFrame(rows)


,account,deal_id,deal,created
0,Citrus Cloud,DL-3001,Platform Subscription,2025-07-01
1,Echo Finance,DL-3005,API Access,2025-09-19


## What’s next
- Swap these crisp Cypher templates with **AIVIA’s NL → FAISS → path → Cypher** pipeline.
- Add an optional `(:Contact)-[:PLAYS_ROLE {role}]->(:Deal)` to model buying committees per-deal.
- Add `StageHistory` for exact stage-age.
- Record a 60–90s Product Hunt video using Question #1 (wow), then #2 (committee gap).

Tip: To reset the demo, just re-run the load cell; MERGE keeps it idempotent.


In [15]:
from pathlib import Path
import sys
# ensure src/ on path
ROOT = Path.cwd().resolve()
SRC = ROOT.parent / "src" if ROOT.name == "notebooks" else ROOT / "src"
sys.path.insert(0, str(SRC))

from aivia.run_query import run_query

question = "Which open deals > 10k created in the last 60 days have no next meeting in the next 14 days?"
cypher, df, debug = run_query(driver, question)
print("Generated Cypher:\n", cypher)
df


Generated Cypher:
 WITH date(localdatetime()) AS today
MATCH (a:Account)-[:HAS_DEAL]->(d:Deal)
WHERE d.stage <> "Closed Won" AND d.stage <> "Closed Lost"
  AND d.amount > 10000
  AND date(d.created_date) >= today - duration({days: 60})
  AND NOT EXISTS {
    MATCH (d)-[:HAS_ACTIVITY]->(act2:Activity)
    WHERE act2.next_step_date IS NOT NULL
      AND date(act2.next_step_date) <= today + duration({days: 14})
  }
OPTIONAL MATCH (d)-[:OWNED_BY]->(u:User)
RETURN a.name AS account, d.id AS deal_id, d.name AS deal, d.amount AS amount,
       d.stage AS stage, d.created_date AS created, u.name AS owner
ORDER BY amount DESC


,account,deal_id,deal,amount,stage,created,owner
0,Aurora Retail,DL-3002,Enterprise Suite,18000.0,Prospecting,2025-09-05,Avery Shaw
1,NovaSecure,DL-3010,Platform Subscription,18000.0,Legal,2025-09-22,Avery Shaw
2,BlueOak Analytics,DL-3012,Compliance Module,12000.0,Legal,2025-09-15,Jordan Lee


In [16]:
for q in [
    "commit deals this quarter missing finance or security",
    "evaluate stage > 21 days with no activity in 14 days"
]:
    cypher, df, debug = run_query(driver, q)
    display({"question": q, "rows": len(df)}, df.head(10))


{'question': 'commit deals this quarter missing finance or security',
 'rows': 1}

,account,deal_id,deal,gap
0,Aurora Retail,DL-3003,Data Integration,Missing Finance & Missing Security


{'question': 'evaluate stage > 21 days with no activity in 14 days', 'rows': 2}

,account,deal_id,deal,created
0,Citrus Cloud,DL-3001,Platform Subscription,2025-07-01
1,Echo Finance,DL-3005,API Access,2025-09-19
